In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
from time import time
import pyodbc
import os
import tables
import time
from azure.storage.blob import BlobService
import urllib2

### Initialize database parameters (CHANGE THIS)

In [2]:
SERVER_NAME   = 'ENTER SERVER_NAME '
DATABASE_NAME = 'ENTER DATABASE_NAME'
USERID        = 'ENTER USERID'
PASSWORD      = 'ENTER PASSWORD'
DB_DRIVER     = 'SQL Server Native Client 11.0'

STORAGEACCOUNTNAME = "ENTER STORAGE ACCOUNT NAME"
STORAGEKEY = "ENTER STORAGE KEY"

#### Read dataset
Dataset is read from a public blob and copied to a private blob to locad it into SQL DW via Polybase

In [9]:
f = urllib2.urlopen('https://cahandson.blob.core.windows.net/nyctaxi/nyctaxipoint1pct.csv')
taxisample = f.read()
blob_service = BlobService(account_name=STORAGEACCOUNTNAME, account_key=STORAGEKEY)
blob_service.create_container('nyctaxinb')
blob_service.put_block_blob_from_bytes(
    'nyctaxinb',
    'nyctaxipoint1pct.csv',
    taxisample
)

In [ ]:
# Construct the SQL DW Connection string
driver = 'DRIVER={' + DB_DRIVER + '}'
server = 'SERVER=' + SERVER_NAME 
database = 'DATABASE=' + DATABASE_NAME
uid = 'UID=' + USERID 
pwd = 'PWD=' + PASSWORD
CONNECTION_STRING = ';'.join([driver,server,database,uid,pwd, 'Encrypt=yes;TrustServerCertificate=no'])
print CONNECTION_STRING

In [11]:
# Connect to the Database. Autocommit needs to turned on for DDL statements
conn = pyodbc.connect(CONNECTION_STRING)
conn.autocommit=True
cursor = conn.cursor()

#### Construct queries for Polybase

We will create a table to hold the dataset in DW under a specific schema "nyctaxinb". Table name is "nyctaxi". The steps to use Polybase are:

1. Create Database Scoped Credential to store Storage account key
1. Create External datasource (the private blob where we copied the dataset)
1. Create external file format 
1. Create external table specifying the format and data source from above
1. Create SQL DW table from the external table

If you are re-running the creation of all the objects and reloading the DW, you will need to go to SQLDW (Via SQL Server Data Tools or Management studio) and delete the objects by running the following SQL statements:
<code>
drop table nyctaxinb.nyctaxi;

drop external table external_nyctaxi_nb;

DROP EXTERNAL FILE FORMAT nyctaxi_ff ;

drop external data source nyctaxi_nb_storage;

drop database scoped credential AzureStorageCredential;  
</code>


In [12]:
# Create Schema
q_sch = '''CREATE SCHEMA nyctaxinb'''

q_dbsc_tpl = '''
CREATE DATABASE SCOPED CREDENTIAL AzureStorageCredential  
WITH IDENTITY = 'user' , 
Secret = '%s'
'''

q_extds_tpl = '''
CREATE EXTERNAL DATA SOURCE nyctaxi_nb_storage 
WITH
(
    TYPE = HADOOP,
    LOCATION ='wasbs://nyctaxinb@%s.blob.core.windows.net',
    CREDENTIAL = AzureStorageCredential
)
'''

q_extff = '''
CREATE EXTERNAL FILE FORMAT nyctaxi_ff 
WITH 
(   
    FORMAT_TYPE = DELIMITEDTEXT, 
    FORMAT_OPTIONS  
    (
        FIELD_TERMINATOR =',',
        DATE_FORMAT = 'MM/dd/yyyy hh:mm:ss tt',
        USE_TYPE_DEFAULT = TRUE
    )
)
'''

q_exttab = '''
CREATE EXTERNAL TABLE external_nyctaxi_nb
(
    medallion varchar(50) not null,
    hack_license varchar(50) not null,
    vendor_id char(3),
    rate_code char(3),
    store_and_fwd_flag char(3),
    pickup_datetime datetime not null,
    dropoff_datetime datetime not null,
    passenger_count int,
    trip_time_in_secs bigint,
    trip_distance float,
    pickup_longitude varchar(30),
    pickup_latitude varchar(30),
    dropoff_longitude varchar(30),
    dropoff_latitude varchar(30),
    payment_type char(3),
    fare_amount float,
    surcharge float,
    mta_tax float,
    tolls_amount float,
    total_amount float,
    tip_amount float,
    tipped int,
    tip_class int
)
with (
    LOCATION    = '.',
    DATA_SOURCE = nyctaxi_nb_storage,
    FILE_FORMAT = nyctaxi_ff,
    REJECT_TYPE = VALUE,
    REJECT_VALUE = 12     
)  
'''

q_createtab = '''
CREATE TABLE nyctaxinb.nyctaxi
WITH 
(   
    CLUSTERED COLUMNSTORE INDEX,
    DISTRIBUTION = HASH(medallion)
)
AS 
SELECT * 
FROM   external_nyctaxi_nb
'''
q_dbsc = q_dbsc_tpl % (STORAGEKEY)
q_extds = q_extds_tpl % (STORAGEACCOUNTNAME)
#cursor.execute(q_sch)
#cursor.commit()
print q_sch
print q_dbsc
print q_extds
print q_extff
print q_exttab
print q_createtab

CREATE SCHEMA nyctaxinb

CREATE DATABASE SCOPED CREDENTIAL AzureStorageCredential  
WITH IDENTITY = 'user' , 
Secret = 'ilbthcJqBHA+NY8yn7EsAuC/PaDfcuZT7DLvylMhwR+gpuY7LThDvU0BtIOYVjRRLTob8etEeOi3iQg37NkNtA=='


CREATE EXTERNAL DATA SOURCE nyctaxi_nb_storage 
WITH
(
    TYPE = HADOOP,
    LOCATION ='wasbs://nyctaxinb@gkgarage.blob.core.windows.net',
    CREDENTIAL = AzureStorageCredential
)


CREATE EXTERNAL FILE FORMAT nyctaxi_ff 
WITH 
(   
    FORMAT_TYPE = DELIMITEDTEXT, 
    FORMAT_OPTIONS  
    (
        FIELD_TERMINATOR =',',
        DATE_FORMAT = 'MM/dd/yyyy hh:mm:ss tt',
        USE_TYPE_DEFAULT = TRUE
    )
)


CREATE EXTERNAL TABLE external_nyctaxi_nb
(
    medallion varchar(50) not null,
    hack_license varchar(50) not null,
    vendor_id char(3),
    rate_code char(3),
    store_and_fwd_flag char(3),
    pickup_datetime datetime not null,
    dropoff_datetime datetime not null,
    passenger_count int,
    trip_time_in_secs bigint,
    trip_distance float,
    pickup_long

#### Now execute the queries constructed above

In [7]:
cursor.execute(q_sch)

ProgrammingError: ('42S01', "[42S01] [Microsoft][SQL Server Native Client 11.0][SQL Server]There is already an object named 'nyctaxinb' in the database.\r\nCREATE SCHEMA failed due to previous errors. (2714) (SQLExecDirectW)")

In [13]:
cursor.execute(q_dbsc)

In [14]:
cursor.execute(q_extds)

In [15]:
cursor.execute(q_extff)

In [16]:
cursor.execute(q_exttab)

In [17]:
cursor.execute(q_createtab)

#### Verify the data is loaded

In [18]:
cursor.close()
pd.read_sql('select top 10 * from nyctaxinb.nyctaxi',conn)

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tolls_amount,total_amount,tip_amount,tipped,tip_class
0,B505BF57EE08B47D1A9A28F746C46A37,0263C0794089506BC7EE5039A788A23F,VTS,1,"""""",2013-03-04 22:27:00,2013-03-04 22:31:00,2,240,1.57,...,40.746799,CRD,6.5,0.5,0.5,0,8.90,1.40,1,1
1,8602C683EF20BE73168B5B4657F655A2,F0CB659C0B6D2FABC574090DC60051B3,VTS,1,"""""",2013-11-06 22:29:00,2013-11-06 22:37:00,6,480,1.66,...,40.773579,CRD,8.0,0.5,0.5,0,11.12,2.12,1,1
2,6B5219D6D71C99D9F17746F1C8C67861,C47850DF4AE043EC699F5BB4D424499D,VTS,1,"""""",2013-09-15 08:04:00,2013-09-15 08:30:00,3,1560,11.55,...,40.678944,CRD,34.0,0.0,0.5,0,43.00,8.50,1,2
3,E807403B8C181E55DFFE25489CC025D2,BF51059B67A9314E4BBCEE6720598847,CMT,1,N,2013-04-13 02:54:09,2013-04-13 03:04:43,1,633,1.90,...,40.707375,CRD,9.5,0.5,0.5,0,12.60,2.10,1,1
4,11DE35431DD1EB8FB3C5982F874A34F8,0F9A7E0BA925C549E69BCE407D912CAD,CMT,1,N,2013-09-04 09:39:42,2013-09-04 09:45:59,1,376,1.20,...,40.763786,CRD,6.5,0.0,0.5,0,8.40,1.40,1,1
5,32BFF45F9BD84ED81A355C98944C175A,7B6D0CD3F60DE976EC588ED4D178B52D,CMT,1,N,2013-07-02 16:28:14,2013-07-02 16:33:18,1,304,0.80,...,40.76899,CRD,5.5,1.0,0.5,0,8.40,1.40,1,1
6,6D42AA76826708EA462441CB7133374C,0C3549B549D90B1DCE1D47F441C5B275,CMT,1,N,2013-08-21 10:50:20,2013-08-21 11:03:11,1,771,2.40,...,40.743935,CSH,10.5,0.0,0.5,0,11.00,0.00,0,0
7,30DAA9127DA29636A5D3CDD370FFB8C2,2C3784F1C3FCE38CDE7BECACB63795CE,CMT,1,N,2013-01-19 06:42:57,2013-01-19 06:48:12,2,314,1.50,...,40.743885,CRD,7.0,0.0,0.5,0,9.00,1.50,1,1
8,57E0853CE50CAEBBB249D42C5D96FD4C,2942468A99C4DE447A339010883272F9,VTS,1,"""""",2013-10-26 07:48:00,2013-10-26 07:56:00,1,480,1.50,...,40.722511,CRD,8.0,0.0,0.5,0,9.50,1.00,1,1
9,97576A28817DE822E51EE06A8F39B121,5AB656A83A7F3A6FB748083983725F18,VTS,1,"""""",2013-01-30 14:26:00,2013-01-30 14:35:00,1,540,0.49,...,40.757805,CRD,7.0,0.0,0.5,0,9.60,2.10,1,1
